In [81]:
import mido 
from mido import MidiFile
import py_midicsv as pm

import numpy as np
import pandas as pd

In [3]:
msg = mido.Message('note_on', note=60)
msg.copy(channel=2)

Message('note_on', channel=2, note=60, velocity=64, time=0)

In [46]:
md = MidiFile('/home/ivan/Desktop/Notes/MIDI/1.mid')

In [47]:
md.tracks[0][-4:]

MidiTrack([
  Message('note_on', channel=0, note=81, velocity=0, time=151),
  Message('note_on', channel=0, note=82, velocity=80, time=9),
  Message('note_on', channel=0, note=82, velocity=0, time=1823),
  MetaMessage('end_of_track', time=1)])

In [48]:
track = md.tracks[0]

In [69]:
track[-3]

Message('note_on', channel=0, note=82, velocity=80, time=9)

In [104]:
def get_midi_data(track):
    df = pd.DataFrame(columns=['note', 'velocity', 'time'])
    i = 0
    for msg in track:
        if msg.type == 'note_on':
            df.loc[i] = [msg.note, msg.velocity, msg.time]
            i += 1
    
    return df

In [129]:
df = get_midi_data(track)
df = df[df['velocity'] != 0].reset_index().drop(columns=['index'])
df.shape

(108, 3)

In [130]:
get_midi_data(track)[:10]

,note,velocity,time
0,75,80,0
1,75,0,479
2,79,80,1
3,79,0,479
4,82,80,1
5,82,0,479
6,81,80,1
7,81,0,479
8,82,80,1
9,82,0,1823
